# DBSCAN : Density Based Spatial Clustering of Applications With Noise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN


In [ ]:
# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]

X, labels_true = make_blobs(n_samples   = 750, 
                            centers     = centers, 
                            cluster_std = 0.4,
                            random_state= 0)

In [ ]:
X[:3]

In [ ]:
plt.plot(X[:,0],X[:,1],'.');

In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
labels_true

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=.3, min_samples=10).fit(X)

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# labels for all the rows
print(labels)

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print(n_clusters_)

In [ ]:
print('Estimated number of clusters: %d'    % n_clusters_)
print("Silhouette Coefficient      : %0.3f" % metrics.silhouette_score(X, labels))

In [ ]:
# Plot result

# Black removed and is used for noise instead.
unique_labels = set(labels)

colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

**Parameters:**

__eps__ : float, optional
The maximum distance between two samples for one to be considered as in the neighborhood of the other. This is not a maximum bound on the distances of points within a cluster. This is the most important DBSCAN parameter to choose appropriately for your data set and distance function.

__min_samples__ : int, optional
The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

__metric__ : string, or callable
The metric to use when calculating distance between instances in a feature array. If metric is a string or callable, it must be one of the options allowed by sklearn.metrics.pairwise_distances for its metric parameter. 

__algorithm__ : {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, optional
The algorithm to be used by the NearestNeighbors module to compute pointwise distances and find nearest neighbors. See NearestNeighbors module documentation for details.

__leaf_size__ : int, optional (default = 30)
Leaf size passed to BallTree or cKDTree. This can affect the speed of the construction and query, as well as the memory required to store the tree. The optimal value depends on the nature of the problem.

__p__ : float, optional
The power of the Minkowski metric to be used to calculate distance between points.

**Attributes:**

__core_sample_indices_ __: array, shape = [n_core_samples] - Indices of core samples.

__components_ __: array, shape = [n_core_samples, n_features]
Copy of each core sample found by training.

__labels_ __: array, shape = [n_samples]
Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1.

In [ ]:
# define hyper parameters
eps_space = np.arange(0.1, 5, 0.1)
min_samples_space = np.arange(1, 25, 1)

In [ ]:
from collections import Counter 
# Example on counter
z = ['blue', 'red', 'blue', 'yellow', 'blue', 'red'] 

col_count = Counter(z) 

print(col_count) 

In [ ]:
%%time
# Looping over each combination of hyperparameters

dbscan_clusters = []

# Starting a tally of total iterations
n_iterations = 0

for eps_val in eps_space:
    for samples_val in min_samples_space:    
        
        # instantiate DBSCAN
        dbscan = DBSCAN(eps = eps_val, min_samples = samples_val)
        
        # fit & predict
        
        # fit()) 	Perform DBSCAN clustering from features or distance matrix.
        # fit_predict() 	Performs clustering on X and returns cluster labels.
        clusters = dbscan.fit_predict(X = X)
        
        labels = dbscan.labels_

        # Counting the amount of data in each cluster
        cluster_count = Counter(clusters)
        
        # Saving the number of clusters
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        
        if n_clusters_ <= 1:
            dbscan_clusters.append([eps_val, samples_val, -11])
            continue;
        
        # Increasing the iteration tally with each run of the loop
        n_iterations += 1
        
        sil_score = metrics.silhouette_score(X, labels)
        
        dbscan_clusters.append([eps_val, samples_val, sil_score])

In [ ]:
results = np.array(dbscan_clusters)

results_df = pd.DataFrame(results, columns=['eps', 'min_points', 'silhouette_score'])

results_df.shape

In [ ]:
results_df

In [ ]:
# sort on the silhouette_score
results_df_sorted = results_df.sort_values(['silhouette_score'], ascending=False)
results_df_sorted

# Using the best parameters {eps, min_samples}

In [ ]:
# instantiate DBSCAN
dbscan = DBSCAN(eps = 0.4, min_samples = 23)

# fit & predict

# fit()) 	Perform DBSCAN clustering from features or distance matrix.
# fit_predict() 	Performs clustering on X and returns cluster labels.
clusters = dbscan.fit_predict(X = X)

labels = dbscan.labels_

In [ ]:
core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print(n_clusters_)

In [ ]:
# Plot result
plt.figure(figsize=(16, 8))

# Black removed and is used for noise instead.
unique_labels = set(labels)

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for k, col in zip(unique_labels, colors):
    
    # Black used for noise.
    if k == -1:
        col = [0, 0, 0, .0001]

    class_member_mask = (labels == k)

    # for the given label and pick the core samples
    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], 
             xy[:, 1], 
             'o', 
             markerfacecolor=col,
             markeredgecolor='k',
             markersize=8)

    # for the given label and pick the non core samples (outliers)
    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], 
             xy[:, 1], 
             'o', 
             markerfacecolor=col,
             markeredgecolor='k', 
             markersize=8)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
print('Estimated number of clusters: %d'    % n_clusters_)
print("Silhouette Coefficient      : %0.3f" % metrics.silhouette_score(X, labels))

**Silhouette Score:** 

The __silhouette score__ is calculated utilizing the mean intra- cluster distance between points, AND the mean nearest-cluster distance. For instance, a cluster with a lot of data points very close to each other (high density) AND is far away from the next nearest cluster (suggesting the cluster is very unique in comparison to the next closest), will have a strong silhouette score. A silhouette score ranges from -1 to 1, with -1 being the worst score possible and 1 being the best score. Silhouette scores of 0 suggest overlapping clusters.

**Inertia:**

__Inertia__ measures the internal cluster sum of squares (sum of squares is the sum of all residuals). Inertia is utilized to measure how related clusters are amongst themselves, the lower the inertia score the better. HOWEVER, it is important to note that inertia heavily relies on the assumption that the clusters are convex (of spherical shape). DBSCAN does not necessarily divide data into spherical clusters, therefore inertia is not a good metric to use for evaluating DBSCAN models Inertia is more often used in other clustering methods, such as K-means clustering.